In [0]:
!pip install tabulate

In [0]:
import pandas as pd
import numpy as np
import os
import csv
import seaborn as sns
import matplotlib.pyplot as plt 
import sklearn as sk
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.decomposition import PCA
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix, confusion_matrix, mean_squared_error
from xgboost import XGBClassifier, XGBRegressor
from sklearn import metrics 
from timeit import default_timer as timer
from google.colab import drive

In [0]:
drive.mount('/Downloads/')

Drive already mounted at /Downloads/; to attempt to forcibly remount, call drive.mount("/Downloads/", force_remount=True).


In [0]:
os.environ['KMP_DUPLICATE_LIB_OK']='True'

train = pd.read_csv('train_no_missing_outlier.csv')

# splitting X and Y from train dataset
X_train = np.array(train.drop(['id','open_date', 'caller_sum_45', 'caller_sum_456',
                              'record_sum_45', 'record_sum_456', 'time_sum_45', 'time_sum_456', 'flag'], axis = 1))
Y_train = np.array(train['flag'])

X_train, X_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=0.2, random_state=0)

weight2 = 0.8
weight1 = 0.2

start = timer()
xgb_model = XGBClassifier(scale_pos_weight = weight2/weight1)
xgb_model.fit(X_train, y_train)

y_pred = xgb_model.predict(X_test)
end = timer()

print("time to run the model: ", end-start)
print(" * Accuracy score for the model: ", xgb_model.score(X_test, y_test))
print(" * Confusion matrix for the model: ", confusion_matrix(y_test, y_pred))

('time to run the model: ', 725.2800641059875)
(' * Accuracy score for the model: ', 0.8230699414175308)
(' * Confusion matrix for the model: ', array([[746397, 126434],
       [ 53690,  91531]]))


In [0]:
print('the roc score is:',roc_auc_score(y_test, y_pred))

('the roc score is:', 0.7427162777697047)


In [0]:
#try different weight for class 0 and 1
num_matrix = 5

confusion_matrix_list = np.zeros((num_matrix, 4, 2)) 
# +2 for including the corresponding weight assignment and accuracy

roc_list = np.zeros(num_matrix)

weight1 = 0.1
weight2 = 0.9
for i in range(num_matrix):
  xgb_model = XGBClassifier(scale_pos_weight = weight2/weight1)
  xgb_model.fit(X_train, y_train)
  y_pred = xgb_model.predict(X_test)
  conf_matrix = confusion_matrix(y_test, y_pred)

  confusion_matrix_list[i][0:-2] = conf_matrix
  confusion_matrix_list[i][-2] = np.array([weight1, weight2])
  roc_score = roc_auc_score(y_test, xgb_model.predict(X_test))
  confusion_matrix_list[i][-1] = np.array([xgb_model.score(X_test, y_test), roc_score])
  weight1 += 0.1
  weight2 -= 0.1
  

In [0]:
# from tabulate import tabulate
# print tabulate(confusion_matrix_list, headers=['True Negative/False Negative','False Positive/True Positive', 'class 0 weight/class 1 weight', 'test accuracy/ ROC score'])

In [0]:
#reduced 2 features (from 12 to 10 features)
os.environ['KMP_DUPLICATE_LIB_OK']='True'

train = pd.read_csv('train_no_missing_outlier.csv')

# splitting X and Y from train dataset
X_train = np.array(train[['day','record_sum_5','caller_sum_5','record_sum_6','record_sum_4',
                               'brand','time_sum_5','time_sum_4','caller_sum_4','city_flag']])
Y_train = np.array(train['flag'])

X_train, X_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=0.2, random_state=0)

weight1 = 0.2
weight2 = 0.8

#start timer
start = timer()
xgb_model = XGBClassifier(scale_pos_weight = weight2/weight1)
xgb_model.fit(X_train, y_train)

y_pred = xgb_model.predict(X_test)
end = timer()
print("time to run the model: ", end-start)
print(" * Accuracy score for the model: ", xgb_model.score(X_test, y_test))
print(" * Confusion matrix for the model: ", confusion_matrix(y_test, y_pred))

('time to run the model: ', 394.6491858959198)
(' * Accuracy score for the model: ', 0.8229805550207652)
(' * Confusion matrix for the model: ', array([[746344, 126487],
       [ 53728,  91493]]))


In [0]:
print('the roc score is:',roc_auc_score(y_test, y_pred))

('the roc score is:', 0.7425550817228371)
